In [1]:
from class_definitions import *

from vivino_db import database
db = database.db
db.db_engine.table_names()

Engine(sqlite://///Users/schlinkertc/code/wine_project/wine/vivino_db/vivino_wines.db)


['regions', 'styles', 'vintages', 'wineries', 'wines']

In [2]:
import pandas as pd
df = pd.read_csv('dataset.csv')

In [3]:
tasting_notes=joblib.load('Wines/tasting_notes.pkl')

In [6]:
t = tasting_notes[0]

In [7]:
t

{4470482: [{'index': 0,
   'tag': 'blackcurrant',
   'note': '2 mentions of black fruit notes'},
  {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
  {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
  {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
  {'index': 3, 'tag': 'chard', 'note': '1 mentions of vegetal notes'},
  {'index': 3, 'tag': 'red beet', 'note': '1 mentions of vegetal notes'},
  {'index': 4, 'tag': 'chocolate', 'note': '1 mentions of oaky notes'}]}

In [9]:
d = {}
for k,v in t.items():
    d['id']=k
    d['notes']=list({x['note'] for x in v})

In [10]:
d

{'id': 4470482,
 'notes': ['2 mentions of black fruit notes',
  '1 mentions of dried fruit notes',
  '1 mentions of floral notes',
  '1 mentions of oaky notes',
  '1 mentions of vegetal notes']}

In [5]:
[{'id':k,'notes':v} for k,v in tasting_notes[0].items()]

[{'id': 4470482,
  'notes': [{'index': 0,
    'tag': 'blackcurrant',
    'note': '2 mentions of black fruit notes'},
   {'index': 0, 'tag': 'plum', 'note': '2 mentions of black fruit notes'},
   {'index': 1, 'tag': 'prune', 'note': '1 mentions of dried fruit notes'},
   {'index': 2, 'tag': 'violet', 'note': '1 mentions of floral notes'},
   {'index': 3, 'tag': 'chard', 'note': '1 mentions of vegetal notes'},
   {'index': 3, 'tag': 'red beet', 'note': '1 mentions of vegetal notes'},
   {'index': 4, 'tag': 'chocolate', 'note': '1 mentions of oaky notes'}]}]